In [1]:
import pandas as pd
import re
import numpy as np
from fuzzywuzzy import fuzz

In [2]:
clubs = pd.read_csv('club_info.csv', index_col=0)
rankings = pd.read_csv('rankings.csv', index_col=0)

In [3]:
pd.set_option('display.max_rows', None)

In [4]:
clubs.head()

,club_name,club_loc,club_id,club_zone
0,209 Water Polo Club,Primary Pool:\r\n\t\t\t\tBuhach Colony High Sc...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 31318\n \r...,Central California Zone
1,A-Town Water Polo Club,Primary Pool:\r\n\t\t\t\tAtascadero High Schoo...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 30234\n \r...,Central California Zone
2,Alumni Water Polo Club,Primary Pool:\r\n\t\t\t\tGranite Bay High Scho...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 23883\n \r...,Central California Zone
3,American River Water Polo Club,Primary Pool:\r\n\t\t\t\tRio Americano High Sc...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 85\n \r\n\...,Central California Zone
4,Arroyo Grande Water Polo,Primary Pool:\r\n\t\t\t\tArroyo Grande High Sc...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 21063\n \r...,Central California Zone


In [5]:
rankings.head()

,Finish,Team,Age,Sex,Year
0,1,LAGUNA BEACH WATER POLO,10,Girls,2016
1,1,VANGUARD AQUATICS,12,Boys,2016
2,1,LOS ANGELES PREMIER,14,Boys,2016
3,1,STANFORD WATER POLO FOUNDATION,16,Boys,2016
4,1,REGENCY WATER POLO CLUB,18,Boys,2016


## Cleaning club data

In [6]:
clubs[clubs['club_name'].str.contains('Test', flags=re.I)]

,club_name,club_loc,club_id,club_zone
117,Test Club 4,Primary Pool:\r\n\t\t\t\tCitrus College\r\n\t\...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 34056\n \r...,Coastal California Zone
242,Test Club 3,Primary Pool:\r\n\t\t\t\tMountain View High Sc...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 17901\n \r...,Northeast Zone
330,Test Club 3,Primary Pool:\r\n\t\t\t\tMountain View High Sc...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 34435\n \r...,Pacific Southwest Zone
331,Test Club 3,Primary Pool:\r\n\t\t\t\tMountain View High Sc...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 34434\n \r...,Pacific Southwest Zone
504,testingclub22,Primary Pool:\r\n\t\t\t\tTBD\r\n\t\t\t\tPacifi...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 34313\n \r...,Southern Pacific Zone


In [7]:
clubs = clubs[~clubs['club_name'].str.contains('Test', flags=re.I)].copy()

In [8]:
clubs['club_id'] = clubs['club_id'].str.replace('Club ID:','').str.strip()
clubs['club_id'] = clubs['club_id'].str.replace('\r','').str.strip()
clubs['club_id'] = clubs['club_id'].str.replace('\n','').str.strip()
clubs['club_id'] = clubs['club_id'].str.replace('\t','').str.strip()
clubs.replace(u'\xa0',u'', regex=True, inplace=True)
clubs.replace(u'\r',u'', regex=True, inplace=True)
clubs.replace(u'\n',u'', regex=True, inplace=True)

In [9]:
clubs[clubs['club_name'].str.contains('Navy')]

,club_name,club_loc,club_id,club_zone
231,Navy Aquatic Club,Primary Pool:\t\t\t\tNaval Academy-Scott Nat\t...,200,Northeast Zone


In [10]:
clubs['og_loc'] = clubs['club_loc']
clubs['pool_name'] = clubs['club_loc'].str.split('\t\t\t\t', ).str.get(1)

In [11]:
clubs['club_loc'] = clubs['club_loc'].str.split('\t\t\t\t', ).str.get(2)

In [12]:
clubs['club_loc'] = clubs['club_loc'].str.replace('(\d+St)', lambda m: m.group(1).lower())
clubs['club_loc'] = clubs['club_loc'].str.replace('(\d+Nd)', lambda m: m.group(1).lower())
clubs['club_loc'] = clubs['club_loc'].str.replace('(\d+Rd)', lambda m: m.group(1).lower())
clubs['club_loc'] = clubs['club_loc'].str.replace('(\d+Th)', lambda m: m.group(1).lower())

clubs['club_loc'] = clubs['club_loc'].str.replace('M/S NG27', '')
clubs['club_loc'] = clubs['club_loc'].str.replace('.', '')
clubs['club_loc'] = clubs['club_loc'].str.replace(' ,', ',')
clubs['club_loc'] = clubs['club_loc'].str.replace('\([\w\s]+\)', '')

clubs['club_loc'] = clubs['club_loc'].str.replace('Dr, ', 'Dr')
clubs['club_loc'] = clubs['club_loc'].str.replace('DR', 'Dr')
clubs['club_loc'] = clubs['club_loc'].str.replace('RD', 'Rd')
clubs['club_loc'] = clubs['club_loc'].str.replace('Blvd ', 'Blvd')
clubs['club_loc'] = clubs['club_loc'].str.replace('Rd ', 'Rd')
clubs['club_loc'] = clubs['club_loc'].str.replace('RdSw', 'Rd Sw')
clubs['club_loc'] = clubs['club_loc'].str.replace('BlvdSw', 'Blvd Sw')
clubs['club_loc'] = clubs['club_loc'].str.replace('Dr ', 'Dr')
clubs['club_loc'] = clubs['club_loc'].str.replace('St, ', 'St')
clubs['club_loc'] = clubs['club_loc'].str.replace('St ', 'St')
clubs['club_loc'] = clubs['club_loc'].str.replace('StW', 'St W')
clubs['club_loc'] = clubs['club_loc'].str.replace('St Wi', 'StWi')
clubs['club_loc'] = clubs['club_loc'].str.replace('Lincoln ', 'Lincoln')
clubs['club_loc'] = clubs['club_loc'].str.replace('OPlaine ', 'Oplaine')
clubs['club_loc'] = clubs['club_loc'].str.replace('SMain St Walnut', 'S Main StWalnut')
clubs['club_loc'] = clubs['club_loc'].str.replace('MacDonough Hall', '121 Blake Rd')

clubs['club_loc'] = clubs['club_loc'].str.replace('PO', 'Po')
clubs['club_loc'] = clubs['club_loc'].str.replace('US', 'Us')
clubs['club_loc'] = clubs['club_loc'].str.replace('NW', 'Nw')
clubs['club_loc'] = clubs['club_loc'].str.replace('NE', 'Ne')
clubs['club_loc'] = clubs['club_loc'].str.replace('SW', 'Sw')
clubs['club_loc'] = clubs['club_loc'].str.replace('SE', 'Se')

clubs['club_loc'] = clubs['club_loc'].str.replace('&', ' ')
clubs['club_loc'] = clubs['club_loc'].str.replace('-', ' ')
clubs['club_loc'] = clubs['club_loc'].str.replace('/', ' ')
clubs['club_loc'] = clubs['club_loc'].str.replace('\/', ' ')
clubs['club_loc'] = clubs['club_loc'].str.replace(':', ' ')
clubs['club_loc'] = clubs['club_loc'].str.replace('\'', '')
clubs['club_loc'] = clubs['club_loc'].str.replace('#', '')

In [13]:
clubs['address'] = clubs['club_loc'].str.extract('(.+?)(?=\B[A-Z,])')

In [14]:
clubs['address'] = clubs['address'].str.replace('North East Corner Of The Palm Desert Civic Center', '43900 San Pablo Ave')

In [15]:
clubs['club_loc'] = clubs['club_loc'].str.replace('ESeattle', 'eSeattle')
clubs['club_loc'] = clubs['club_loc'].str.replace('ESeattle', 'eSeattle')
clubs['club_loc'] = clubs['club_loc'].str.replace('NSeattle', 'nSeattle')
clubs['club_loc'] = clubs['club_loc'].str.replace('ESalt', 'eSalt')
clubs['club_loc'] = clubs['club_loc'].str.replace('2700 ESlc', 'eCottonwood Heights')
clubs['club_loc'] = clubs['club_loc'].str.replace('WKearns', 'wKearns')
clubs['club_loc'] = clubs['club_loc'].str.replace('WBountiful', 'wBountiful')
clubs['club_loc'] = clubs['club_loc'].str.replace('WUniversity', 'wUniversity')
clubs['club_loc'] = clubs['club_loc'].str.replace('McAlister', 'Mcalister')
clubs['club_loc'] = clubs['club_loc'].str.replace('MacDonough', 'Macdonough')
clubs['club_loc'] = clubs['club_loc'].str.replace('MacArthur', 'Macarthur')
clubs['club_loc'] = clubs['club_loc'].str.replace('WayPo', 'Way Po')
clubs['club_loc'] = clubs['club_loc'].str.replace('StLouis', 'St Louis')
clubs['club_loc'] = clubs['club_loc'].str.replace('StGeorge', 'St George')
clubs['club_loc'] = clubs['club_loc'].str.replace('StCharles', 'St Charles')
clubs['club_loc'] = clubs['club_loc'].str.replace('StPeters', 'St Peters')
clubs['club_loc'] = clubs['club_loc'].str.replace('thousand', 'Thousand')
clubs['club_loc'] = clubs['club_loc'].str.replace('BlvdSw', 'Blvd Sw')

In [16]:
clubs['state'] = clubs['club_loc'].str.extract('([A-Z]{2})')

clubs['city'] = clubs['club_loc'].str.extract('[a-z0-9]([A-Z][a-z]+\s*\w*\s*\w*),')

In [17]:
clubs.replace(u'\t',u'', regex=True, inplace=True)
clubs.replace(u'\r',u'', regex=True, inplace=True)
clubs.replace(u'\n',u'', regex=True, inplace=True)

In [18]:
clubs['city'] = clubs['city'].str.replace('NeSeattle', 'Seattle')
clubs['city'] = clubs['city'].str.replace('Central High SchoolErie', 'Erie')
clubs['state'] = clubs['state'].str.replace('OP', 'RD')

In [19]:
clubs['zip'] = np.nan
clubs['zip'] = clubs['club_loc'].str.extract('([0-9]{5})\s?[0-9]*$')

In [20]:
clubs.iloc[250, 4]

'Primary Pool:Roosevelt High School1120 Nehoa StreetHonolulu, HI96822'

In [21]:
clubs[clubs['city'].str.contains('SwAlbany', na=False, flags=re.I)]

,club_name,club_loc,club_id,club_zone,og_loc,pool_name,address,state,city,zip


In [22]:
clubs[clubs['city'].isnull()]

,club_name,club_loc,club_id,club_zone,og_loc,pool_name,address,state,city,zip
131,Calgary Masters,NaN,19401,International Zone,Unknown,NaN,NaN,NaN,NaN,NaN
132,Calgary Polo Splash,NaN,17376,International Zone,Unknown,NaN,NaN,NaN,NaN,NaN
212,Barcelona International Water Polo Association,NaN,34590,Northeast Zone,Unknown,NaN,NaN,NaN,NaN,NaN
394,Soda Moms,NaN,34704,Pacific Zone,Unknown,NaN,NaN,NaN,NaN,NaN


In [23]:
clubs = clubs.drop(['club_loc'], axis=1)

In [24]:
clubs.loc[clubs['address'] == '1800 Buhach Road', 'zip'] = '95301'

In [25]:
clubs['loc'] = clubs['address'] + ", " + clubs['city'] + ", " + clubs['state'] + " " + clubs['zip']

In [26]:
clubs.shape

(535, 10)

In [27]:
clubs[clubs['loc'].isnull()]

,club_name,club_id,club_zone,og_loc,pool_name,address,state,city,zip,loc
131,Calgary Masters,19401,International Zone,Unknown,NaN,NaN,NaN,NaN,NaN,NaN
132,Calgary Polo Splash,17376,International Zone,Unknown,NaN,NaN,NaN,NaN,NaN,NaN
212,Barcelona International Water Polo Association,34590,Northeast Zone,Unknown,NaN,NaN,NaN,NaN,NaN,NaN
394,Soda Moms,34704,Pacific Zone,Unknown,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
clubs = clubs.fillna('Unknown')

In [29]:
clubs['club_name'] = clubs['club_name'].str.replace(',', '').str.strip()
clubs['club_name'] = clubs['club_name'].str.replace('.', '').str.strip()
clubs['club_name'] = clubs['club_name'].str.replace(')', '').str.strip()
clubs['club_name'] = clubs['club_name'].str.replace('(', '').str.strip()

clubs['club_name'] = clubs['club_name'].str.lstrip()
clubs['club_name'] = clubs['club_name'].str.upper().str.strip()

In [30]:
remove_wp = ['AQUATICS', 'AQUATIC', 'WATER POLO', 'CLUB', 'FEDERATION', 'TEAM', 'WP', 'WPC', 'WATERPOLO', 'POLO', 'POL', 'AQ', 'SUMMER', 
           'ORGANIZATION', 'ACADEMY', 'ORG', 'SWIM', 'AC', 'INC', 'FOUNDATION']
pat_wp = r'\b(?:{})\b'.format('|'.join(remove_wp))
clubs['club_short'] = clubs['club_name'].str.replace(pat_wp, '').str.strip()

In [31]:
clubs.loc[clubs['club_name'].str.contains('ACADEMY WATER POLO'), 'club_short'] = 'ACADEMY'
clubs.loc[clubs['club_name'] == 'EAST', 'club_short'] = 'EAST'
clubs['club_short'] = clubs['club_short'].replace('\s+', ' ', regex=True)

In [32]:
clubs.head()

,club_name,club_id,club_zone,og_loc,pool_name,address,state,city,zip,loc,club_short
0,209 WATER POLO CLUB,31318,Central California Zone,Primary Pool:Buhach Colony High School1800 Buh...,Buhach Colony High School,1800 Buhach Road,CA,Atwater,95301,"1800 Buhach Road, Atwater, CA 95301",209
1,A-TOWN WATER POLO CLUB,30234,Central California Zone,Primary Pool:Atascadero High School1 High Scho...,Atascadero High School,1 High School Hill,CA,Atascadero,93422,"1 High School Hill, Atascadero, CA 93422",A-TOWN
2,ALUMNI WATER POLO CLUB,23883,Central California Zone,Primary Pool:Granite Bay High School Pool1 Gri...,Granite Bay High School Pool,1 Grizzly Way,CA,Granite Bay,95746,"1 Grizzly Way, Granite Bay, CA 95746",ALUMNI
3,AMERICAN RIVER WATER POLO CLUB,85,Central California Zone,Primary Pool:Rio Americano High School4540 Ame...,Rio Americano High School,4540 American River Dr,CA,Sacramento,95864,"4540 American River Dr, Sacramento, CA 95864",AMERICAN RIVER
4,ARROYO GRANDE WATER POLO,21063,Central California Zone,Primary Pool:Arroyo Grande High School495 Vall...,Arroyo Grande High School,495 Valley Rd,CA,Arroyo Grande,93420,"495 Valley Rd, Arroyo Grande, CA 93420",ARROYO GRANDE


In [33]:
clubs[clubs['club_short'].isnull()]

,club_name,club_id,club_zone,og_loc,pool_name,address,state,city,zip,loc,club_short


In [34]:
clubs = clubs[clubs['club_name']!='BULLDOG WATER POLO CLUB']
clubs = clubs[clubs['club_name']!='LONG BEACH WATER POLO CLUB']
clubs = clubs[clubs['club_name']!='ORANGE COUNTY WATER POLO FOUNDATION']

In [35]:
clubs[clubs['club_short'].duplicated(keep=False)]

,club_name,club_id,club_zone,og_loc,pool_name,address,state,city,zip,loc,club_short


## Cleaning ranking data

In [36]:
rankings['Sex'] = rankings['Sex'].str.replace('Bcoys','Boys')
rankings['Sex'] = rankings['Sex'].str.replace('ip','Boys')
rankings['Sex'] = rankings['Sex'].fillna('Mixed')

In [37]:
rankings['Team'] = rankings['Team'].str.strip(r'\\r')
rankings['Team'] = rankings['Team'].str.replace('\t\r', '')
rankings['Team'] = rankings['Team'].str.replace('\r', ' ')

In [38]:
rankings['og_name'] = rankings['Team']

In [39]:
rankings['Squad'] = rankings['Team'].str.extract(r'\b([A-F])\b')
rankings['Squad'] = rankings['Squad'].fillna('A')

In [40]:
rankings['Team'] = rankings['Team'].str.replace('ORAGNE', 'ORANGE')

In [41]:
remove_words = ['[0-9]{2}', '[UBG][0-9]{2}', '[0-9]{2}[UBGS]' '20[0-9]{2}' 'GIRL', 'BOY', 'COED',
                'GIRLS', 'BOYS', 'COEDS', 'JO', 'JOQ', 'JOS', '[A-E]', '&', 'AND', 'UNDER', 'BLACK', 'GREEN', 'BLUE', 
                'BLU', 'RED', 'WHITE', 'YELLOW', 'PINK', 'MAROON', 'GOLD', 'SILVER', 'BRONZE', 'PURPLE', 'BLK', 'GREY', 'U', 'B', 'G',]

pattern = r'\b(?:{})\b'.format('|'.join(remove_words))
rankings['Team'] = rankings['Team'].str.replace(pattern, '')

In [42]:
rankings['Team'] = rankings['Team'].str.replace('\s+', ' ', regex=True)
rankings['Team'] = rankings['Team'].str.replace(r'\(TIED.+\)', '')
rankings['Team'] = rankings['Team'].str.replace(r'\(TIED.+', '')
rankings['Team'] = rankings['Team'].str.replace(r"[\"\'.,\\\(\)&/]", '')
rankings['Team'] = rankings['Team'].str.replace('\t\r', '')
rankings['Team'] = rankings['Team'].str.replace('xad', '')
rankings['Team'] = rankings['Team'].str.replace('\\xad', '')
rankings['Team'] = rankings['Team'].str.replace('‐', '')
rankings['Team'] = rankings['Team'].str.replace('-', '')
rankings['Team'] = rankings['Team'].str.replace(r'[0-9]{2}[UBGS]', '')
rankings['Team'] = rankings['Team'].str.replace(r'20[0-9]{2}', '')
rankings['Team'] = rankings['Team'].str.replace('ORANGE COUNTY', 'ORANGECOUNTY')
rankings['Team'] = rankings['Team'].str.replace(r'\b(ORANGE)\b', '')
rankings['Team'] = rankings['Team'].str.replace('NORTHWOOD NAVY', 'NORTHWOOD')
rankings['Team'] = rankings['Team'].str.replace('CHAWPWHITE', 'CHAWP')
rankings['Team'] = rankings['Team'].str.replace('IRLS', '')
rankings['Team'] = rankings['Team'].str.replace('\s+', ' ', regex=True)
rankings['Team'] = rankings['Team'].str.strip()

In [43]:
rankings.loc[rankings['og_name'].str.contains('TIED'), 'Tie'] = 1
rankings['Tie'] = rankings['Tie'].fillna(0)

In [44]:
rankings.columns = ['finish', 'club_name', 'age', 'sex', 'year', 'og_name', 'squad', 'tie']

In [45]:
rankings['club_short'] = rankings['club_name'].str.replace(pat_wp, '').str.strip()
rankings.loc[rankings['club_name'].str.contains('ACADEMY WATER POLO'), 'club_short'] = 'ACADEMY'
rankings.loc[rankings['club_name'] == 'EAST', 'club_short'] = 'EAST'
rankings.loc[rankings['club_name'].str.contains('GOLD COAST'), 'club_short'] = 'GOLD COAST'
rankings.loc[rankings['club_name'].str.contains('NWCWATER'), 'club_short'] = 'NWC'
rankings.loc[rankings['club_name'].str.contains('CDM'), 'club_short'] = 'CDM'
rankings['club_short'] = rankings['club_short'].str.lstrip()

In [46]:
rankings.head()

,finish,club_name,age,sex,year,og_name,squad,tie,club_short
0,1,LAGUNA BEACH WATER POLO,10,Girls,2016,LAGUNA BEACH WATER POLO,A,0.0,LAGUNA BEACH
1,1,VANGUARD AQUATICS,12,Boys,2016,VANGUARD AQUATICS,A,0.0,VANGUARD
2,1,LOS ANGELES PREMIER,14,Boys,2016,LOS ANGELES PREMIER,A,0.0,LOS ANGELES PREMIER
3,1,STANFORD WATER POLO FOUNDATION,16,Boys,2016,STANFORD WATER POLO FOUNDATION,A,0.0,STANFORD
4,1,REGENCY WATER POLO CLUB,18,Boys,2016,REGENCY WATER POLO CLUB,A,0.0,REGENCY


## Matching club names

In [47]:
clubs[clubs['club_name'].str.contains('DACA')]

,club_name,club_id,club_zone,og_loc,pool_name,address,state,city,zip,loc,club_short


In [48]:
data = rankings.merge(clubs[['club_short', 'club_id', 'club_zone', 'loc', 'state']], on='club_short', how='left')

In [49]:
data[data['club_id'].notnull()].shape

(4206, 13)

In [50]:
data.head()

,finish,club_name,age,sex,year,og_name,squad,tie,club_short,club_id,club_zone,loc,state
0,1,LAGUNA BEACH WATER POLO,10,Girls,2016,LAGUNA BEACH WATER POLO,A,0.0,LAGUNA BEACH,717,Southern Pacific Zone,"625 Park Ave, Laguna Beach, CA 92651",CA
1,1,VANGUARD AQUATICS,12,Boys,2016,VANGUARD AQUATICS,A,0.0,VANGUARD,17185,Southern Pacific Zone,"2301 W Mac, Santa Ana, CA 92704",CA
2,1,LOS ANGELES PREMIER,14,Boys,2016,LOS ANGELES PREMIER,A,0.0,LOS ANGELES PREMIER,22230,Coastal California Zone,"3700 Coldwater Cayon, Studio City, CA 91604",CA
3,1,STANFORD WATER POLO FOUNDATION,16,Boys,2016,STANFORD WATER POLO FOUNDATION,A,0.0,STANFORD,570,Pacific Zone,"Avery Aquatic Center, Stanford, CA 94305",CA
4,1,REGENCY WATER POLO CLUB,18,Boys,2016,REGENCY WATER POLO CLUB,A,0.0,REGENCY,NaN,NaN,NaN,NaN


In [51]:
data.shape

(6255, 13)

In [52]:
rankings.shape

(6255, 9)

In [53]:
data[data['club_id'].isnull()].shape

(2049, 13)

In [54]:
missing_clubs = data[data['club_id'].isnull()].copy()

### Fuzzy matching club names

In [55]:
def match_name(name, list_names, min_score=0):
    max_score = -1
    max_name = ""
    for name2 in list_names:
        score = fuzz.ratio(name, name2)
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

In [56]:
dict_list = []
for club in missing_clubs['club_short']:
    match = match_name(club, clubs['club_short'], 75)
    
    dict_ = {}
    dict_.update({"club_name" : club})
    dict_.update({"match_name" : match[0]})
    dict_.update({"match amount" : match[1]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)
merge_table.head()

,club_name,match_name,match amount
0,REGENCY,,-1
1,TROJAN CA,TROJAN CA,95
2,ORANGECOUNTY,ORANGE COUNTY,96
3,ORANGECOUNTY,ORANGE COUNTY,96
4,ORANGECOUNTY,ORANGE COUNTY,96


In [57]:
map1 = merge_table[merge_table['match_name'] != ''].copy()

In [58]:
map1.drop_duplicates(subset=['club_name'], inplace=True)

In [59]:
map1.shape

(99, 3)

In [60]:
map1.head()

,club_name,match_name,match amount
1,TROJAN CA,TROJAN CA,95
2,ORANGECOUNTY,ORANGE COUNTY,96
11,SPARTAN MI,SPARTAN MI,91
16,INTERNATIONAL,ORLANDO INTERNATIONAL,76
22,IRVINE KAHUNA,IRVINE KAHUNA,93


In [61]:
map2 = merge_table[merge_table['match_name'] == ''].copy()
map2.drop_duplicates(subset=['club_name'], inplace=True)

In [62]:
map2.head()

,club_name,match_name,match amount
0,REGENCY,,-1
5,DACA,,-1
6,LA JOLLA,,-1
14,JESUIT,,-1
20,SAN MIGUEL,,-1


In [63]:
correct_clubs = {'LA JOLLA':'LA JOLLA UNITED', 'LA JOLLA S':'LA JOLLA UNITED', 'SET':'SADDLEBACK EL TORO SET', 
                 'DIABLO':'DIABLO ALLIANCE', 'GULLIVER RIPTIDES':'MIAMI RIPTIDES', 'NSD STARS':'NORTH SAN DIEGO STARS', 
                 'SCV':'SANTA CLARA', 'CMAC ALL STARS':'COSTA MESA', 'SHAQ':'SLEEPY HOLLOW', 'BIG VALLEY':'UNIVERSITY PACIFIC',
                'STOCKTON':'UNIVERSITY PACIFIC', 'SOAK':'SONORA', 'VIPER PIGEON':'WEST HOUSTON', 'NORTHWOOD':'NORTH IRVINE',
                'SPARTAN':'SPARTAN MI', 'LWP':'LAMORINDA', 'SCOTS':'SAINT ANDREW\'S', 'WEST SUB NAPERVILLE':'WEST SUBURBAN',
                'HERIMAN':'SOUTH COUNTY-HERRIMAN', 'KAHUNA':'IRVINE KAHUNA', 'SD SHORES':'SAN DIEGO SHORES', 'CORONA DEL MAR':'CDM',
                'SO PASADENA':'SOUTH PASADENA', 'ORWP':'OLIMPIAKAN RESERVNER ORWP', 'OLIMPIAKAN ORWP':'OLIMPIAKAN RESERVNER ORWP', 
                 '680 DRIVER':'680', 'TROJAN':'TROJAN CA', 'WEST SUBURBAN NAPERVILLE':'WEST SUBURBAN', 'ESCONDIDO RIPTIDES':'NORTH COUNTY RIPTIDES',
                'THILLS':'TUALATIN HILLS', 'TM VEGASHENDERSON':'VEGAS', 'VEGASHENDERSON':'VEGAS', 'TM VEGAS HEND':'VEGAS',
                'CC SHARKS':'SANTA CRUZ', 'CENTRAL COAST SHARKS':'SANTA CRUZ', 'GREENWICH WATER':'GREENWICH', 'MERCED':'209',
                'ATWATERMERCED':'209', 'VIPER PIGEONS':'WEST HOUSTON', 'VIPER PIGEON SOUTH':'WEST HOUSTON', 'VIPER PIGEON NORTH':'WEST HOUSTON',
                'VIPER PIGEON NORTH WEST':'WEST HOUSTON', 'VIPER PIGEON WEST':'WEST HOUSTON', 'VIPER PIGEON SOUTH EAST':'WEST HOUSTON',
                'T HILLS':'TUALATIN HILLS', 'SB PREMIER':'SANTA BARBARA PREMIER', 'ORWP TITANS':'OLIMPIAKAN RESERVNER ORWP',
                'HOUSTON STORM':'NORTH HOUSTON', 'SD SHROES':'SAN DIEGO SHORES', 'SB 805':'SANTA BARBARA 805', '680 DRIVERS':'680',
                'ATWATER MERCED':'209', 'LA JOLLA   S':'LA JOLLA UNITED', 'W SUB NAPERVILLE':'WEST SUBURBAN', 'MODESTON STAN':'MODESTO/STANISLAUS',
                'MODESTO STAN':'MODESTO/STANISLAUS', 'AM RIVER':'AMERICAN RIVER', 'AM RIV':'AMERICAN RIVER', 'HERRIMAN':'SOUTH COUNTY-HERRIMAN',
                'REGENCU':'REGENCY', 'ORCUTT ASSOCIATIONONE WAY':'ORCUTT ASSOCIATION/ONE WAY', 'OLIMPIKIAN ORWP':'OLIMPIAKAN RESERVNER ORWP',
                'ORCUTT':'ORCUTT ASSOCIATION/ONE WAY', 'NORTH DALLAS SPARTANS':'TRIDENT', 'DALLAS':'NORTH TEXAS THUNDER',
                'ST MARKS LIONS':'TRIDENT', 'NORTH DALLAS SPARTAN':'TRIDENT', 'NORTH SHORE':'INGLEMOOR HIGH SCHOOL', 'HOUSTON':'WEST HOUSTON',
                'IRVINE KAHUNAWEST COAST':'IRVINE KAHUNA', 'SOUTHSIDE':'SOUTHSIDE HONEY BADGERS', 'TROJAN CARDINAL':'TROJAN CA', 
                'ANN ARBOR':'WOLVERINE', 'NOTHERN ELITE':'ELITE', 'EAST VALE':'CIRCLE CITY', 'EASTVALE':'CIRCLE CITY', 'PALOS VERDES':'PV',
                'HB':'HUNTINGTON BEACH', 'HUNTINGTON BEACH G':'HUNTINGTON BEACH', 'SHORES SOLANA':'SAN DIEGO SHORES', 'HBWPC':'HUNTINGTON BEACH',
                '616':'WOLVERINE', 'OC RIPTIDE':'PACIFIC COAST RIPTIDE', 'WEST COAST AQAUTICS':'WEST COAST', 'ORCA':'VILLA PARK', 'CHULA VISTA':'SOL',
                'ORCUTT ASSOCIATION':'ORCUTT ASSOCIATION/ONE WAY', 'EXPO SHARKS':'LOS ANGELES', 'SBWPC':'SANTA BARBARA 805',
                'LOS AL':'LOS ALAMITOS', 'PRINCETON':'TIGER', 'LAGUNA':'LAGUNA BEACH', 'RIPTIDES':'MIAMI RIPTIDES', 
                'ALLIANCE':'CAPISTRANO VALLEY ALLIANCE', 'DACA':'222', 'ANTEATER':'IRVINE KAHUNA', 'CVAC':'CAPISTRANO VALLEY ALLIANCE',
                'NORTHWOOD HS':'NORTH IRVINE', 'TPC SHARKS':'TPC', 'MVPOLO':'MID VALLEY', 'NAPA VALLEY':'NAPA PREMIER',
                'NAPA':'NAPA PREMIER', 'SD SHORES 680':'SAN DIEGO SHORES', 'PLACER COUNTY':'NORTHERN SIERRA',
                'SACTO':'SACRAMENTO', 'BRENTWOOD SEAWOLVES':'LAMORINDA BRENTWOOD', 'CMAC':'COSTA MESA', 'BALL':'BALL UNDER',
                'VC PREMIER': 'GOLD COAST', 'SHARKS':'SANTA CRUZ', 'ST LOUIS AREA SLAP':'SLAP', 'SHARKS M':'SANTA CRUZ',
                'SOAC':'SOAC UCLS', 'SOVA':'SOUTH VALLEY', 'LAWPC':'LOS ANGELES', 'ORANGECOUNTY RIPTIDE':'ORANGE COUNTY',
                'THUNDER':'NORTH TEXAS THUNDER', 'VC UNITED':'GOLD COAST', 'NAPA VALLEY':'NAPA PREMIER', 'NORTH BEACH':'NORTH PALM BEACH',
                'LA PREMIER':'LOS ANGELES PREMIER', 'COAST':'GOLD COAST', 'OCWPC':'ORANGE COUNTY', 'BIG VALLEY':'UNIVERSITY PACIFIC',
                'SAN DIEGO INTERNATIONAL':'GREATER SAN DIEGO', 'CAPITAL PREMIER':'CAPITAL DC', 'NWPC':'NORTHWEST ORCAS', 'NORTHWEST':'NORTHWEST ORCAS',
                'CAL':'CAL BEARS', 'CLOVIS BEARS':'CLOVIS', 'LAZERS':'LARPD', 'CENTRAL COAST':'SANTA ANA', 'HANFORD':'KINGS',
                'SFWPC':'SAN FRANCISCO WARRIORS', 'WILLAMETTE':'WILLAMETTE VALLEY',
                'NORCAL':'NORCAL - NORTHERN CALIFORNIA', 'PUGET':'PUGET SOUND', 'POWAY':'POWAY VALLEY', 'NO ILL PC':'NIPC', 'NO ILLINOIS':'NIPC',
                'NORTHERN SAN DIEGO NEPTUNE':'NORTH SAN DIEGO STARS', 'CTRL COAST SHARKS':'SANTA CRUZ', 'CLOVIS COUGARS':'CLOVIS',
                'WILAMETTE':'WILLAMETTE VALLEY', 'GILRS14':'', 'CRUSH':'NORTHWEST BLUE CRUSH', 'SJWPF':'SAN JOSE', 'BEND':'BEND WAVES',
                'SARASOTA UNITED':'SOUTHWEST FLORIDA', 'VANGUARD SENIOR':'VANGUARD', 'SFLA HURRICANS':'SOUTH FLORIDA - HURRICANES',
                'SFLA HURRICANES':'SOUTH FLORIDA - HURRICANES', 'CAPITAL AREA PREMIER CAP':'CAPITAL DC', 'VALLEY FALCONS OYS':'VALLEY FALCONS',
                'SAN MIGUEL':'LA JOLLA UNITED', 'DIRTY BAY':'WEST HOUSTON', 'RENEGADES':'REDLANDS RENEGADES', 'SV OUTLAW':'SILICON VALLEY OUTLAWS',
                'ONE WAY':'ORCUTT ASSOCIATION/ONE WAY', 'AGWP':'ARROYO GRANDE', 'AGWP NAVY':'ARROYO GRANDE', 'SOQUEL WAXEM':'SOQUEL', 'PNW UNITED':'PNW SHORES', 
                'ELITE RIVERSIDE':'RIVERSIDE', 'CAL REPUBLIC':'CALIFORNIA REPUBLIC SC','RANCHO SAN DIEGO NORSEMEN':'ODIN', 'PAC NW UNITED':'PNW SHORES', 'SAN MIGUEL WPF':'SAN DIEGO SHORES',
                'SBWPF':'SANTA BARBARA 805', 'SONS OF ODIN':'ODIN', 'STARS STRIPES':'BRUIN', 'CHICAGO PARK':'CHICAGO PARK DISTRICT',
                'LIGHTNING':'IRVINE KAHUNA', 'BANGARANG':'NAPA PREMIER', 'SBU':'SOUTH BAY UNITED', 'COLLEGE PARK S':'ACADEMY', 'LANCER':'SEASIDE',
                'CERRITOS':'LOS ALAMITOS', 'ECWP':'EUGENE CITY', 'SJ WPF':'SAN JOSE', 'SJA':'SAN JOSE/ALMADEN', 'SJ ALMADEN':'SAN JOSE/ALMADEN',
                'VIKINGS':'DYNAMO', 'CLOVIS NORTH':'CLOVIS', 'ARC':'AMERICAN RIVER', 'BLUECRUSH':'NORTHWEST BLUE CRUSH', 'ORLANDO':'ORLANDO UNITED',
                'SOCAL FHS':'SOCAL', 'SEASIDE NAVY':'SEASIDE', 'THE SANTA BARBARA PALS':'SANTA BARBARA PREMIER', 'SALT LAKE POSEIDON':'UTAH EAST SIDE',
                'BACK BAY KAHUNA':'BACK BAY', 'KAHUNA GREENWICH':'IRVINE KAHUNA', 'MICHIGAN':'MID-MICHIGAN RAPTOR', 'SO BAY PENIN':'SOUTH BAY UNITED',
                'BAY AREA SUNSET':'BAY AREA', 'GIG HARBOR':'NARROWS', 'DWPC AGGIES':'DAVIS', 'GREAT LAKES':'MID-MICHIGAN RAPTOR', 'SOLANO':'SOLANO COUNTY',
                'SCSC 18A':'SANTA CLARA', 'SCSC':'SANTA CLARA', 'NYAC':'NEW YORK ATHLETIC'}

In [64]:
rankings[rankings['club_short'].str.contains('LIGHTNING')]

,finish,club_name,age,sex,year,og_name,squad,tie,club_short
390,35,LIGHTNING,16,Boys,2011,LIGHTNING,A,0.0,LIGHTNING
395,36,LIGHTNING,14,Boys,2011,LIGHTNING,A,0.0,LIGHTNING
480,47,LIGHTNING,18,Boys,2011,LIGHTNING,A,0.0,LIGHTNING


In [65]:
clubs[clubs['club_name'].str.contains('LIGHTNING')]

,club_name,club_id,club_zone,og_loc,pool_name,address,state,city,zip,loc,club_short


# Merge updated rankings table 

In [66]:
new_clubs = [pd.Series(['BALL UNDER', 0, 'Northeast Zone', 'Wakefield Swimming Pool, 4901 S. Chesterfield Road, Arlington, VA 22206', 'Wakefield Swimming Pool', '4901 S. Chesterfield Road', 'VA', 'Arlington', '22206', '4901 S Chesterfield Road, Arlington, VA 22206', 'BALL UNDER'], index=clubs.columns),
            pd.Series(['ELLIS', 0, 'Central California Zone', 'Sierra High School, 1700 Thomas St, Manteca, CA 95337', 'Sierra High School', '1700 Thomas St', 'CA', 'Manteca', '95337', '1700 Thomas St, Manteca, CA 95337', 'ELLIS'], index=clubs.columns),
            pd.Series(['BUENA', 0, 'Coastal California Zone', 'Buena High School, 5670 Telegraph Rd, Ventura, CA 93003', 'Buena High School', '5670 Telegraph Rd', 'CA', 'Ventura', '93003', '5670 Telegraph Rd, Ventura, CA 93003', 'BUENA'], index=clubs.columns),
            pd.Series(['SLAP', 0, 'Midwest Zone', 'Chaminade High School, 425 S Lindbergh Blvd St. Louis, MO 63131', 'Chaminade High School', '425 S Lindbergh Blvd', 'MO', 'St Louis', '63131', '425 S Lindbergh Blvd, St Louis, MO 63131', 'SLAP'], index=clubs.columns),
            pd.Series(['SOUTH PASADENA', 0, 'Coastal California Zone', 'South Pasadena High School, 1401 Fremont Ave, South Pasadena, CA 91030', 'South Pasadena High School', '1401 Fremont Ave', 'CA', 'South Pasadena', '91030', '1401 Fremont Ave, South Pasadena, CA 91030', 'SOUTH PASADENA'], index=clubs.columns),
            pd.Series(['LODI', 0, 'Central California Zone', 'Lodi High School, 3 S Pacific Ave, Lodi, CA 95242', 'Lodi High School', '3 S Pacific Ave', 'CA', 'Lodi', '95242', '3 S Pacific Ave, Lodi, CA 95242', 'LODI'], index=clubs.columns),
            pd.Series(['DESERT', 0, 'Southern Pacific Zone', '14048 Cobalt Rd, Victorville, CA 92392', 'Unknown', '14048 Cobalt Rd', 'CA', 'Victorville', '92392', '14048 Cobalt Rd, Victorville, CA 92392', 'DESERT'], index=clubs.columns),
            pd.Series(['WEST COAST', 0, 'Pacific Zone', '1350 Bacchus Dr, San Jose, CA 95122', 'Unknown', '1350 Bacchus Dr', 'CA', 'San Jose', '95122', '1350 Bacchus Dr, San Jose, CA 95122', 'WEST COAST'], index=clubs.columns),
            pd.Series(['XTREME', 0, 'Southern Pacific Zone', 'CBU, 8432 Magnolia Ave, Riverside, CA 92504', 'CBU', '8432 Magnolia Ave', 'CA', 'Riverside', '92504', '8432 Magnolia Ave, Riverside, CA 92504', 'XTREME'], index=clubs.columns),
            pd.Series(['REGENCY', 0, 'Southern Pacific Zone', 'Mater Dei High, 1202 W Edinger Ave, Santa Ana, CA 92707', 'Mater Dei High', '1202 W Edinger Ave', 'CA', 'Santa Ana', '92707', '1202 W Edinger Ave, Santa Ana, CA 92707', 'REGENCY'], index=clubs.columns),
            pd.Series(['STRAKE JESUIT', 0, 'Southwest Zone', 'Strake Jesuit College Preparatory, 8900 Bellaire Blvd, Houston, TX 77036', 'Strake Jesuit College Preparatory', '8900 Bellaire Blvd', 'TX', 'Houston', '77036', '8900 Bellaire Blvd, Houston, TX 77036', 'JESUIT'], index=clubs.columns),
            pd.Series(['GOLDEN STATE', 0, 'Pacific Zone', '1700 W Hillsdale Blvd, San Mateo, CA 94402', 'Unknown', '1700 W Hillsdale Blvd', 'CA', 'San Mateo', '94402', '1700 W Hillsdale Blvd, San Mateo, CA 94402', 'GOLDEN STATE'], index=clubs.columns),
            pd.Series(['SCRIPPS', 0, 'Pacific Southwest Zone', '10410 Falcon Way, San Diego, CA 92131', 'Unknown', '10410 Falcon Way', 'CA', 'San Diego', '92131', '10410 Falcon Way, San Diego, CA 92131', 'SCRIPPS'], index=clubs.columns),
            pd.Series(['LAMORINDA BRENTWOOD', 0, 'Central California Zone', '101 American Ave, Brentwood, CA 94513', 'Unknown', '101 American Ave', 'CA', 'Brentwood', '94513', '101 American Ave, Brentwood, CA 94513', 'LAMORINDA BRENTWOOD'], index=clubs.columns),
            pd.Series(['ROWLAND RACERS', 0, 'Coastal California Zone', 'Rowland, High School, 2000 S Otterbein Ave, Rowland Heights, CA 91748', 'Rowland High School', '2000 S Otterbein Ave', 'CA', 'Rowland Heights', '91748', '2000 S Otterbein Ave, Rowland Heights, CA 91748', 'RACERS'], index=clubs.columns), 
            pd.Series(['ACADEMY WATER POLO CLUB', 0, 'Pacific Zone', '147 Gregory Ln, Pleasant Hill, CA 94523', 'Unknown', '147 Gregory Ln', 'CA', 'Pleasant Hill', '94523', '147 Gregory Ln, Pleasant Hill, CA 94523', 'ACADEMY'], index=clubs.columns),
            pd.Series(['OHLONE', 0, 'Pacific Zone', '43600 Mission Blvd, Fremont, CA 94539', 'Unknown', '43600 Mission Blvd', 'CA', 'Fremont', '94539', '43600 Mission Blvd, Fremont, CA 94539', 'OHLONE'], index=clubs.columns), 
            pd.Series(['VALLEY FALCONS', 0, 'Southern Pacific Zone', '1801 S Greenville St, Santa Ana, CA 92704', 'Unknown', '1801 S Greenville St', 'CA', 'Santa Ana', '92704', '1801 S Greenville St, Santa Ana, CA 92704', 'VALLEY FALCONS'], index=clubs.columns), 
            pd.Series(['FULLERTON HILLS', 0, 'Southern Pacific Zone', '1801 Lancer Way, Fullerton, CA 92833', 'Unknown', '1801 Lancer Way', 'CA', 'Fullerton', '92833', '1801 Lancer Way, Fullerton, CA 92833', 'FULLERTON HILLS'], index=clubs.columns),
            pd.Series(['PALM SPRINGS', 0, 'Coastal California Zone', '2401 E Baristo Rd, Palm Springs, CA 92262', 'Palm Springs High School', '2401 E Baristo Rd', 'CA', 'Palm Springs', '92262', '2401 E Baristo Rd, Palm Springs, CA 92262', 'PALM SPRINGS'], index=clubs.columns),
            pd.Series(['PIERCE COUNTY', 0, 'Pacific Northwest & Hawaii Zone', '5410 20th St E, Fife, WA 98424', 'Unknown', '5410 20th St E', 'WA', 'Fife', '98424', '5410 20th St E, Fife, WA 98424', 'PIERCE COUNTY'], index=clubs.columns),
            pd.Series(['LA CANADA', 0, 'Southern Pacific Zone', '4463 Oak Grove Dr, La Canada Flintridge, CA 91011', 'Unknown', '4463 Oak Grove Dr', 'CA', 'La Canada Flintridge', '91011', '4463 Oak Grove Dr, La Canada Flintridge, CA 91011', 'LA CANADA'], index=clubs.columns),
            pd.Series(['PNW SHORES', 0, 'Pacific Northwest & Hawaii Zone', '650 SW Campus Dr, Federal Way, WA 98023', 'Unknown', '650 SW Campus Dr', 'WA', 'Federal Way', '98023', '650 SW Campus Dr, Federal Way, WA 98023', 'PNW SHORES'], index=clubs.columns),
            pd.Series(['SAN MARINO', 0, 'Southern Pacific Zone', '2701 Huntington Dr, San Marino, CA 91108', 'San Marino High School', '2701 Huntington Dr', 'CA', 'San Marino', '91108', '2701 Huntington Dr, San Marino, CA 91108', 'SAN MARINO'], index=clubs.columns),
            pd.Series(['SANTA ANA', 0, 'Southern Pacific Zone', '520 West Walnut, Santa Ana, CA 92701', 'Santa Ana High School', '520 West Walnut', 'CA', 'Santa Ana', '92701', '520 West Walnut, Santa Ana, CA 92701', 'SANTA ANA'], index=clubs.columns),
            pd.Series(['LOS ANGELES ATHLETIC LAAC', 0, 'Southern Pacific Zone', '431 W 7th St, Los Angeles, CA 90014', 'Unknown', '431 W 7th St', 'CA', 'Los Angeles', '90014', '431 W 7th St, Los Angeles, CA 90014', 'LOS ANGELES ATHLETIC LAAC'], index=clubs.columns),
            pd.Series(['SOQUEL', 0, 'Pacific Zone', '401 Soquel San Jose Rd, Soquel, CA 95073', 'Unknown', '401 Soquel San Jose Rd', 'CA', 'Soquel', '95073', '401 Soquel San Jose Rd, Soquel, CA 95073', 'SOQUEL'], index=clubs.columns)]

In [67]:
clubs = clubs.append(new_clubs, ignore_index=True) 

In [68]:
clubs.head()

,club_name,club_id,club_zone,og_loc,pool_name,address,state,city,zip,loc,club_short
0,209 WATER POLO CLUB,31318,Central California Zone,Primary Pool:Buhach Colony High School1800 Buh...,Buhach Colony High School,1800 Buhach Road,CA,Atwater,95301,"1800 Buhach Road, Atwater, CA 95301",209
1,A-TOWN WATER POLO CLUB,30234,Central California Zone,Primary Pool:Atascadero High School1 High Scho...,Atascadero High School,1 High School Hill,CA,Atascadero,93422,"1 High School Hill, Atascadero, CA 93422",A-TOWN
2,ALUMNI WATER POLO CLUB,23883,Central California Zone,Primary Pool:Granite Bay High School Pool1 Gri...,Granite Bay High School Pool,1 Grizzly Way,CA,Granite Bay,95746,"1 Grizzly Way, Granite Bay, CA 95746",ALUMNI
3,AMERICAN RIVER WATER POLO CLUB,85,Central California Zone,Primary Pool:Rio Americano High School4540 Ame...,Rio Americano High School,4540 American River Dr,CA,Sacramento,95864,"4540 American River Dr, Sacramento, CA 95864",AMERICAN RIVER
4,ARROYO GRANDE WATER POLO,21063,Central California Zone,Primary Pool:Arroyo Grande High School495 Vall...,Arroyo Grande High School,495 Valley Rd,CA,Arroyo Grande,93420,"495 Valley Rd, Arroyo Grande, CA 93420",ARROYO GRANDE


In [69]:
map1 = map1.drop(['match amount'], axis=1)
map1.set_index('club_name', inplace=True)

In [70]:
map1_dict = map1.to_dict()
map1_dict = map1_dict['match_name']

In [71]:
del map1_dict['WEST COAST']
del map1_dict['BEAST']
del map1_dict['VIKINGS']
del map1_dict['FRASER VALLEY']

In [72]:
final_rankings = rankings.copy()
final_rankings['club_short'] = final_rankings['club_short'].str.upper()
final_rankings['club_short'] = final_rankings['club_short'].replace('\s+', ' ', regex=True)
final_rankings['club_short'].replace(correct_clubs, regex=False, inplace=True)
final_rankings['club_short'].replace(map1_dict, regex=False, inplace=True)
final_rankings['club_short'] = final_rankings['club_short'].str.strip()
final_rankings['club_short'] = final_rankings['club_short'].replace('\s+', ' ', regex=True)

In [73]:
complete_rank = final_rankings.merge(clubs[['club_short', 'club_id', 'club_zone', 'address', 'city', 'zip', 'loc', 'state']], on='club_short', how='left')

In [74]:
complete_rank[complete_rank['club_id'].isnull()].shape

(53, 16)

In [75]:
complete_rank.shape

(6255, 16)

In [76]:
rankings.shape

(6255, 9)

In [77]:
complete_rank.loc[complete_rank['club_id'].isnull(), 'club_short'].value_counts()

                    10
NCA                  4
RR                   4
BEAR                 3
A2                   3
HOLA!                3
SPRING               2
LRP                  2
ONE                  2
FRASER VALLEY        2
BYE                  1
BEAST                1
NXT                  1
WOODCREEK            1
TWWPMAROON           1
ERHS                 1
NORSE                1
OCEANSIDE            1
SERPENT WARRIORS     1
TWWP                 1
AVENGERS             1
101                  1
COPS                 1
SPWPC                1
SEASIDE ROYAL        1
FORCE                1
APACHE               1
UNITED COZY          1
Name: club_short, dtype: int64

In [78]:
complete_rank.head()

,finish,club_name,age,sex,year,og_name,squad,tie,club_short,club_id,club_zone,address,city,zip,loc,state
0,1,LAGUNA BEACH WATER POLO,10,Girls,2016,LAGUNA BEACH WATER POLO,A,0.0,LAGUNA BEACH,717,Southern Pacific Zone,625 Park Ave,Laguna Beach,92651,"625 Park Ave, Laguna Beach, CA 92651",CA
1,1,VANGUARD AQUATICS,12,Boys,2016,VANGUARD AQUATICS,A,0.0,VANGUARD,17185,Southern Pacific Zone,2301 W Mac,Santa Ana,92704,"2301 W Mac, Santa Ana, CA 92704",CA
2,1,LOS ANGELES PREMIER,14,Boys,2016,LOS ANGELES PREMIER,A,0.0,LOS ANGELES PREMIER,22230,Coastal California Zone,3700 Coldwater Cayon,Studio City,91604,"3700 Coldwater Cayon, Studio City, CA 91604",CA
3,1,STANFORD WATER POLO FOUNDATION,16,Boys,2016,STANFORD WATER POLO FOUNDATION,A,0.0,STANFORD,570,Pacific Zone,Avery Aquatic Center,Stanford,94305,"Avery Aquatic Center, Stanford, CA 94305",CA
4,1,REGENCY WATER POLO CLUB,18,Boys,2016,REGENCY WATER POLO CLUB,A,0.0,REGENCY,0,Southern Pacific Zone,1202 W Edinger Ave,Santa Ana,92707,"1202 W Edinger Ave, Santa Ana, CA 92707",CA


In [79]:
complete_rank.to_csv('complete_ranking.csv')